In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline
from datetime import datetime
from datetime import date
from datetime import timedelta
import glob
import cx_Oracle

days= mdates.DayLocator() 
cmap=plt.cm.tab20c.colors #https://matplotlib.org/stable/tutorials/colors/colormaps.html
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_columns', 1000)

In [ ]:
class Connection(cx_Oracle.Connection):
  def cursor(self):
    cursor = super(Connection, self).cursor()
    cursor.arraysize = 5000
    return cursor

#dsnStr = cx_Oracle.makedsn("10.145.66.7", "19550", "EPRDOPER")
#connection = Connection(user='WWANALYSIS', password='Info0nly', dsn=dsnStr)

# dsnStr = cx_Oracle.makedsn("10.145.66.59", "19580", "BETAOPER")
# connection = Connection(user='WWANALYSIS', password='Info0nly', dsn=dsnStr)

dsnStr = cx_Oracle.makedsn("r-wwlabdev.win.ebmud", "18000", "WWLABDEV")
connection = Connection(user='LIMSHIST', password='sushi', dsn=dsnStr)

# prints 5000 just to see it's working
print(connection.cursor().arraysize)

In [ ]:
tag_list=['OAK_EST_UP_LVL', 'OAK_EST_DN_LVL']

date_start="""'2025-09-21'"""
datetime_current=pd.Timestamp.now()
date_end="'"+(datetime_current).strftime('%Y-%m-%d %H:%M')+"'"


# Creating string with each tag (as nvarchar2(40) to insert into SQL select )
tag_string=""""""
for tag in tag_list:
    if tag!=tag_list[-1]:
        tag_string+="CAST('"+tag+"' as nvarchar2(40)),"
    else:
        tag_string+="CAST('"+tag+"' as nvarchar2(40))"
 
sql_tag="""SELECT b.\"tag\", b.\"descriptor\" FROM  pipoint@piprd b
WHERE (b.\"tag\" IN ("""+tag_string+""")) """        

sql_df_tag=pd.read_sql_query(sql_tag, connection)

# Confirm correct tags
sql_df_tag

In [ ]:
# Equivalent to PI "sampled" data
# DON'T MODIFY SQL SELECT STATEMENT
sql_interp="""SELECT * FROM  piinterp@piprd b 
WHERE (b.\"tag\" IN ("""+tag_string+""")) 
AND (b.\"time\" >= TO_DATE("""+date_start+""", 'YYYY-MM-DD'))
AND (b.\"time\" <= TO_DATE("""+date_end+""", 'YYYY-MM-DD hh24:mi'))
AND b.\"timestep\" = '1h'"""

sql_df_interp=pd.read_sql_query(sql_interp,con=connection)

In [ ]:
# # PI compressed data
# sql_comp="""SELECT * FROM  picomp@piprd b 
# WHERE (b.\"tag\" IN ("""+tag_string+""")) 
# AND (b.\"time\" >= TO_DATE("""+date_start+""", 'YYYY-MM-DD'))
# AND (b.\"time\" <= TO_DATE("""+date_end+""", 'YYYY-MM-DD'))"""

# # PI averaged data
# sql_avg="""SELECT * FROM  piavg@piprd b 
# WHERE (b.\"tag\" IN ("""+tag_string+""")) 
# AND (b.\"time\" >= TO_DATE("""+date_start+""", 'YYYY-MM-DD'))
# AND (b.\"time\" <= TO_DATE("""+date_end+""", 'YYYY-MM-DD'))
# AND b.\"timestep\" = '1d'"""

# # PI Sampled data
# sql_interp="""SELECT * FROM  piinterp@piprd b 
# WHERE (b.\"tag\" IN ("""+tag_string+""")) 
# AND (b.\"time\" >= TO_DATE("""+date_start+""", 'YYYY-MM-DD'))
# AND (b.\"time\" <= TO_DATE("""+date_end+""", 'YYYY-MM-DD'))
# AND b.\"timestep\" = '1h'"""

In [ ]:
# Creating dataframe with datetime index and each PI tag as its own field
temp_list=[]
df_process=sql_df_interp
for tag in df_process['tag'].unique():
    temp_df=df_process[df_process['tag']==tag]
    temp_df=temp_df[['time','value']]
    temp_df.set_index('time',inplace = True)
    temp_df.columns=[str(tag)]
    temp_df = temp_df.loc[~temp_df.index.duplicated(keep='last')]
    temp_list.append(temp_df.resample('1h').ffill())
    # temp_list.append(temp_df)

df_samp=pd.concat(temp_list,axis=1)

df_samp.columns=sql_df_tag['descriptor'].values
df_samp.index.rename('DateTime',inplace=True)

In [ ]:
df_samp